Essais auto-ML avec **autokeras** sur les image: Inutilisable car beaucoup trop lent sur un simple PC, pas très verbeux

In [ ]:
# a voir: pyCaret, Azure

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from time import time
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import autokeras as ak
import kerastuner

def f1_score_w(y_true, y_pred, **kwargs):
    return f1_score(y_true, y_pred, average='weighted')


def image_path(row, subdir="image_train"):
    """ Construction du chemin d'un fichier image """
    f = "image_%d_product_%d.jpg" % (row.imageid, row.productid) 
    return os.path.join(os.getcwd(), "images", subdir, f)


def dataset_generator(imagepaths, targets, nb):
    """ Fonction génératrice de paires (X, y) """
    for i in range(nb):
        f = imagepaths[i].decode('utf-8')
        img = mpimg.imread(f)
        y = targets[i].reshape(1)
        yield img, y


def get_dataset(nb=-1, batch_size=16):
    """ Dataset de nb échantillons """
    imagepaths = list(pd.read_csv("X_train_update.csv")[:nb].apply(image_path, axis=1))
    prdtypes = list(pd.read_csv("Y_train_CVw08PX.csv")["prdtypecode"][:nb])
    uniqprdtypes = sorted(list(set(prdtypes)))
    prdtype_to_int = {u:i for i, u in enumerate(uniqprdtypes) }                
    targets = np.array([prdtype_to_int[p] for p in prdtypes])
    dataset = tf.data.Dataset.from_generator(dataset_generator, args=[imagepaths, targets, len(imagepaths)],
                                             output_types=(tf.int32, tf.int32),
                                             output_shapes=([500, 500, 3], [1,]))
    #dataset = dataset.shuffle(buffer_size=100000)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(1)
    return dataset


dataset = get_dataset(4000)
#iterator = iter(dataset)
#next_element = iterator.get_next()

# overwitting=False => créé un ./image_classifier et repart de là
clf = ak.ImageClassifier(max_trials=10,
#                         objective=kerastuner.Objective('val_f1_score_w', direction='max'),
#                         metrics=[f1_score_w],
                         seed=123)

t0 = time()
clf.fit(dataset, epochs=4, validation_split=0.15, verbose=2)
print(f"Executé en {int(time()-t0)} secondes")



INFO:tensorflow:Reloading Oracle from existing project .\image_classifier\oracle.json
INFO:tensorflow:Reloading Tuner from .\image_classifier\tuner0.json

Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
image_block_1/b...|efficient         |vanilla           
image_block_1/n...|True              |True              
image_block_1/a...|True              |False             
image_block_1/i...|True              |None              
image_block_1/i...|False             |None              
image_block_1/i...|0                 |None              
image_block_1/i...|0                 |None              
image_block_1/i...|0.1               |None              
image_block_1/i...|0                 |None              
image_block_1/e...|True              |None              
image_block_1/e...|b7                |None              
image_block_1/e...|True              |None              
image_block_1/e...|True              |None              
classification_...|gl